In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from scipy import stats as st

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:

# Global constants. Note that you will change  rare doc frequency threshold and max_features will be programmatically as part of hyperparameter optimization. I've set a default value for testing purposes



ENGLISH_STOP_WORDS_MODIFIED =[  # List taken from sklearn's defaults and adjusted according to manual inspection of data and prior beliefs about predictive words 
    'a',
    'about',
    'above',
    'across',
    'after',
    'afterwards',
    #'again',
    'against',
    'ain',
    'all',
    #'almost',
    #'alone',
    'along',
    #'already',
    #'also',
    #'although',
    'always',
    'am',
    'among',
    'amongst',
    'amoungst',
    'amount',
    'an',
    'and',
    #'another',
    #'any',
    'anyhow',
    #'anyone',
    #'anything',
    'anyway',
    #'anywhere',
    'are',
    'aren',
    'around',
    'as',
    'at',
    'back',
    'be',
    'became',
    'because',
    'become',
    'becomes',
    'becoming',
    'been',
    #'before',
    'beforehand',
    #'behind',
    'being',
    #'below',
    'beside',
    'besides',
    'between',
    #'beyond',
    'bill',
    'both',
    'bottom',
    'but',
    'by',
    #'call',
    'can',
    #'cannot',
    #'cant',
    'co',
    'con',
    #'could',
    #'couldn',
    #'couldnt',
    'cry',
    'd',
    'de',
    #'describe',
    #'detail',
    #'did',
    #'didn',
    'do',
    #'does',
    #'doesn',
    'doing',
    'don',
    #'done',
    'down',
    'due',
    'during',
    'each',
    'eg',
    'eight',
    'either',
    'eleven',
    'else',
    'elsewhere',
    'empty',
    #'enough',
    'etc',
    'even',
    'ever',
    'every',
    'everyone',
    'everything',
    'everywhere',
    #'except',
    #'few',
    'fifteen',
    'fify',
    'fill',
    #'find',
    'fire',
    #'first',
    'five',
    'for',
    #'former',
    #'formerly',
    'forty',
    'found',
    'four',
    'from',
    'front',
    'full',
    #'further',
    'get',
    'give',
    'go',
    'had',
    'hadn',
    #'has',
    #'hasn',
    #'hasnt',
    'have',
    'haven',
    'having',
    'he',
    'hence',
    'her',
    'here',
    'hereafter',
    'hereby',
    'herein',
    'hereupon',
    'hers',
    'herself',
    'him',
    'himself',
    'his',
    'how',
    #'however',
    'hundred',
    'i',
    'ie',
    'if',
    'in',
    'inc',
    'indeed',
    'interest',
    'into',
    'is',
    #'isn',
    'it',
    'its',
    'itself',
    #'just',
    #'keep',
    #'last',
    'latter',
    'latterly',
    #'least',
    #'less',
    'll',
    'ltd',
    'm',
    'ma',
    #'made',
    #'many',
    'may',
    'me',
    'meanwhile',
    'might',
    'mightn',
    'mill',
    'mine',
    #'more',
    #'moreover',
    #'most',
    #'mostly',
    'move',
    'much',
    'must',
    'mustn',
    'my',
    'myself',
    'name',
    #'namely',
    #'needn',
    #'neither',
    #'never',
    #'nevertheless',
    #'next',
    'nine',
    'no',
    #'nobody',
    'none',
    'noone',
    'nor',
    'not',
    #'nothing',
    'now',
    'nowhere',
    'o',
    'of',
    'off',
    'often',
    'on',
    'once',
    'one',
    'only',
    'onto',
    'or',
    'other',
    #'others',
    #'otherwise',
    'our',
    'ours',
    'ourselves',
    'out',
    'over',
    'own',
    'part',
    'per',
    'perhaps',
    'please',
    'put',
    #'rather',
    're',
    's',
    'same',
    #'see',
    #'seemed',
    #'seeming',
    #'seems',
    #'serious',
    #'several',
    'shan',
    'she',
    'should',
    'shouldn',
    'show',
    'side',
    'since',
    'sincere',
    'six',
    'sixty',
    'so',
    'some',
    'somehow',
    'someone',
    'something',
    'sometime',
    'sometimes',
    'somewhere',
    'still',
    'such',
    'system',
    't',
    'take',
    'ten',
    'than',
    'that',
    'the',
    'their',
    'theirs',
    'them',
    'themselves',
    'then',
    'thence',
    'there',
    'thereafter',
    'thereby',
    'therefore',
    'therein',
    'thereupon',
    'these',
    'they',
    'thick',
    'thin',
    'third',
    'this',
    'those',
    'though',
    'three',
    'through',
    'throughout',
    'thru',
    'thus',
    'to',
    'together',
    'too',
    #'top',
    'toward',
    'towards',
    'twelve',
    'twenty',
    'two',
    'un',
    'under',
    'until',
    'up',
    'u',  #added
    'wa', #added
    'upon',
    'us',
    've',
    #'very',
    'via',
    'was',
    'wasn',
    'we',
    'well',
    'were',
    'weren',
    'what',
    'whatever',
    'when',
    'whence',
    'whenever',
    'where',
    'whereafter',
    'whereas',
    'whereby',
    'wherein',
    'whereupon',
    'wherever',
    'whether',
    'which',
    'while',
    'whither',
    'who',
    'whoever',
    'whole',
    'whom',
    'whose',
    'why',
    'will',
    'with',
    'within',
    'without',
    'won',
    #'would',
    #'wouldn',
    'y',
    'yet',
    'you',
    'your',
    'yours',
    'yourself',
    'yourselves'
]

# Note; Need to write a helper function to get the manual features included into the sklearn vectorizer class. Will have to do it in a way
# that allows those features to avoid the cutoffs due to min/max document frequency//top N features. If we really need to,
# we can do a very roundabout way by making another vectorizer that is run only on text with these features in it, then applying
# that vectorizer via the .transform() method to our text data, and then concatenating the datasets.
MANUAL_FEATURES = [
    "cannot", "unconvinced", "needs", "unclear", "already", "how will", "how do", "unlikely", "why don't", "is not novel work", "isn't novel", "is novel work", "novel work", "is novel", "an accept", "a reject", "convincing", "interesting"] # TODO RETURN TO if time
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return ([self.wnl.lemmatize(t) for t in word_tokenize(doc)])
RARE_DOCUMENT_FREQUENCY_THRESHOLD = 1  # For now, will implement pipeline to do hyperparameter optimization on these features
MAX_FEATURES = 2000;  #  For now, will implement pipeline to do hyperparameter optimization on these features


In [ ]:
def read_data(path):
  df = pd.read_csv(path)
  return df

In [ ]:
def preprocess(df, column):
  df = df.dropna(subset=[column]).reset_index(drop=True)
  lemmatizer = WordNetLemmatizer()
  stop_words = set(stopwords.words('english'))

  cleaned = []
  for review in df[column]:
    temp = review.lower()
    temp = temp.replace('\n', ' ')
    temp = temp.replace(',', ' ')
    temp = temp.replace('.', ' ')
    temp = temp.replace('!', ' ')
    temp = temp.replace('?', ' ')
    temp = temp.replace(':', ' ')
    temp = temp.replace(';', ' ')
    temp = [lemmatizer.lemmatize(word) for word in temp.split() if word not in stop_words]
    temp = ' '.join(temp)
    cleaned.append(temp)
  df[column] = cleaned
  return df

In [ ]:
data_as_given = read_data('data_reviews_filtered.csv')
data_as_given = preprocess(data_as_given, 'comments')

In [ ]:
data_as_given['accepted'] = data_as_given['accepted'].astype(str).str.strip().str.capitalize()
data_as_given['accepted'] = data_as_given['accepted'].map({'True': 1, 'False': 0})
data_as_given.head()

In [ ]:
data_as_given["accepted"]

In [ ]:
x_data = data_as_given['comments']
y_data = data_as_given['accepted']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

ct_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(), stop_words=ENGLISH_STOP_WORDS_MODIFIED, min_df=RARE_DOCUMENT_FREQUENCY_THRESHOLD, max_features=MAX_FEATURES, ngram_range=(1,3))
tf_vect = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words = ENGLISH_STOP_WORDS_MODIFIED, min_df=RARE_DOCUMENT_FREQUENCY_THRESHOLD, max_features=MAX_FEATURES, ngram_range=(1,3), use_idf=False)
tfidf_vect = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words = ENGLISH_STOP_WORDS_MODIFIED, min_df=RARE_DOCUMENT_FREQUENCY_THRESHOLD, max_features=MAX_FEATURES, ngram_range=(1,3), use_idf=True)


In [ ]:
def svm_train(df, vectorizer, x_train, x_test, y_train, y_test, C, coef0, degree, gamma, kernel):
  x_train = vectorizer.fit_transform(x_train)
  x_test = vectorizer.transform(x_test)

  svm = SVC(C = C, coef0 = coef0, degree = degree, gamma = gamma, kernel=kernel)
  svm.fit(x_train, y_train)
  y_pred = svm.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)
  print(classification_report(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))

  cv_results_f1 = cross_val_score(svm, x_train, y_train, cv=5, scoring='f1')
  print("CV F1 Score:", cv_results_f1.mean())
  cv_results_acc = cross_val_score(svm, x_train, y_train, cv=5, scoring='accuracy')
  print("CV Accuracy Score:", cv_results_acc.mean())

  return svm

In [ ]:
def rf_train(df, vectorizer, x_train, x_test, y_train, y_test, n_estimators, criterion, max_features, max_depth, bootstrap):
  x_train = vectorizer.fit_transform(x_train)
  x_test = vectorizer.transform(x_test)

  rf = RandomForestClassifier(class_weight='balanced', n_estimators= n_estimators, criterion= criterion, max_features= max_features, max_depth= max_depth, bootstrap= bootstrap)
  rf.fit(x_train, y_train)
  y_pred = rf.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  print(classification_report(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))

  cv_results_f1 = cross_val_score(rf, x_train, y_train, cv=5, scoring='f1')
  print("CV F1 Score:", cv_results_f1.mean())
  cv_results_acc = cross_val_score(rf, x_train, y_train, cv=5, scoring='accuracy')
  print("CV Accuracy Score:", cv_results_acc.mean())

  return rf

In [ ]:
def lr_train(df, vectorizer, x_train, x_test, y_train, y_test, penalty, C, max_iter, solver):
  x_train = vectorizer.fit_transform(x_train)
  x_test = vectorizer.transform(x_test)

  lr = LogisticRegression(penalty = penalty, C = C, max_iter = max_iter, solver = solver)
  lr.fit(x_train, y_train)
  y_pred = lr.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)
  print(classification_report(y_test, y_pred))

  cv_results_f1 = cross_val_score(lr, x_train, y_train, cv=5, scoring='f1')
  print("CV F1 Score:", cv_results_f1.mean())
  cv_results_acc = cross_val_score(lr, x_train, y_train, cv=5, scoring='accuracy')
  print("CV Accuracy Score:", cv_results_acc.mean())

  return lr

In [ ]:
def grid_search(svm, x_train, y_train):
  param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 'gamma': ['scale', 'auto'], 'degree': [2, 3, 4, 5], 'coef0': [0.0, 0.1, 0.5, 1.0]}
  grid_search = GridSearchCV(svm, param_grid, cv=5)

  grid_search.fit(x_train, y_train)
  print("Best parameters:", grid_search.best_params_)

In [ ]:
def grid_search_rf(rf, x_train, y_train):
  param_grid = {
    'n_estimators': [25, 40, 50],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['log2', 'sqrt'],
    'max_depth': [2,4],
    'bootstrap': [True, False]
    }
  grid_search = GridSearchCV(rf, param_grid, cv=5)

  grid_search.fit(x_train, y_train)
  print("Best parameters:", grid_search.best_params_)

In [ ]:
def grid_search_lr(rf, x_train, y_train):
  param_grid = {'penalty':['l2'],'C':[1, 10, 100], 'max_iter': [100, 200, 500, 1000], 'solver': ['lbfgs', 'saga', 'liblinear']}
  grid_search = GridSearchCV(rf, param_grid, cv=5)

  grid_search.fit(x_train, y_train)
  print("Best parameters:", grid_search.best_params_)

In [ ]:
svm_ct = svm_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, 10, 1.0, 2, 'auto', 'poly')
svm_tf = svm_train(data_as_given, tf_vect, x_train, x_test, y_train, y_test, 10, 1.0, 2, 'auto', 'poly')
svm_tfidf = svm_train(data_as_given, tfidf_vect, x_train, x_test, y_train, y_test, 10, 1.0, 2, 'auto', 'poly')

In [ ]:
x_train_ct = ct_vectorizer.fit_transform(x_train)
x_train_tf = tf_vect.fit_transform(x_train)
x_train_idf = tfidf_vect.fit_transform(x_train)

In [ ]:
grid_search(svm_ct, x_train_ct, y_train)
grid_search(svm_tf, x_train_tf, y_train)
grid_search(svm_tfidf, x_train_idf, y_train)

In [ ]:
print("ct")
svm_ct_tuned = svm_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, 10, 1.0, 2, 'auto', 'poly')
print("tf")
svm_tf_tuned = svm_train(data_as_given, tf_vect, x_train, x_test, y_train, y_test, 1, 1.0, 3, 'scale', 'poly')
print("tfidf")
svm_tfidf_tuned = svm_train(data_as_given, tfidf_vect, x_train, x_test, y_train, y_test, 10, 0.0, 2, 'scale', 'poly')

In [ ]:
rf_ct = rf_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, 25, 'gini', 'sqrt', 2, False)
rf_tf = rf_train(data_as_given, tf_vect, x_train, x_test, y_train, y_test, 25, 'gini', 'sqrt', 2, False)
rf_tfidf = rf_train(data_as_given, tfidf_vect, x_train, x_test, y_train, y_test, 25, 'gini', 'sqrt', 2, False)

grid_search_rf(rf_ct, x_train_ct, y_train)
grid_search_rf(rf_tf, x_train_tf, y_train)
grid_search_rf(rf_tfidf, x_train_idf, y_train)

In [ ]:
print("ct")
rf_ct = rf_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, 40, 'gini', 'log2', 4, False)
print("tf")
rf_tf = rf_train(data_as_given, tf_vect, x_train, x_test, y_train, y_test, 40, 'gini', 'sqrt', 2, True)
print("tfidf")
rf_tfidf = rf_train(data_as_given, tfidf_vect, x_train, x_test, y_train, y_test, 50, 'log_loss', 'sqrt', 4, True)

In [ ]:
lr_ct = lr_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, None, 10, 100, 'saga')
lr_tf = lr_train(data_as_given, tf_vect, x_train, x_test, y_train, y_test, None, 10, 100, 'saga')
lr_tfidf = lr_train(data_as_given, tfidf_vect, x_train, x_test, y_train, y_test, None, 10, 100, 'saga')

grid_search_lr(lr_ct, x_train_ct, y_train)
grid_search_lr(lr_tf, x_train_tf, y_train)
grid_search_lr(lr_tfidf, x_train_idf, y_train)

In [ ]:
print("ct")
lr_ct = lr_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, 'l2', 1, 1000, 'saga')
print("tf")
lr_tf = lr_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, 'l2', 10, 200, 'saga')
print("tfidf")
lr_tfidf = lr_train(data_as_given, ct_vectorizer, x_train, x_test, y_train, y_test, 'l2', 10, 100, 'saga')

In [ ]:
coefs=lr_ct.coef_[0]
top_three_ct = np.argpartition(coefs, -3)[-3:]
print(data_as_given.feature_names[top_three])